# Bitcoin price forecasting with PySpark
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



# Dependencies, Libraries and Tools

In [19]:
import pandas as pd
import functools

from google.colab import drive

#  Define metrics and parameters

In this section we are going to define the parameters used to collect the data and the metrics used. I will consider Bitcoin data for 10 years, starting from 2012-01-01 through 2022-12-31.

Note that since the Blockchain.com API allows retreiving data with a maximum timespan equal to 6 years, I manually computed the continue date so that I could make a second API call to get the remaining data.

Regarding the metrics, I chose the ones that seemed most relevant to me, containing both price statistics but also technical features of Bitcoin's blockchain.

In [20]:
# Define the parameters
timespan = "6years" # Duration of the data (because the Max timespan == 6years)
start_date = "2016-01-01"
continue_date = "2021-12-31" # The continue date (manually calculate the continue_date)
end_date = "2023-07-31"

# Metrics considered
metrics = [
          ##Currency Statistics##
            "market-price", # Market Price: The average USD market price across major bitcoin exchanges.
            "trade-volume", #E xchange Trade Volume (USD): The total USD value of trading volume on major bitcoin exchanges.

          ##Block Details##
            "blocks-size", # Blockchain Size (MB): The total size of the blockchain minus database indexes in megabytes.
            "avg-block-size", # Average Block Size (MB): The average block size over the past 24 hours in megabytes.
            "n-transactions-total", # Total Number of Transactions: The total number of transactions on the blockchain.
            "n-transactions-per-block", # Average Transactions Per Block: The average number of transactions per block over the past 24 hours.

          ##Mining Information##
            "hash-rate", # Total Hash Rate (TH/s): The estimated number of terahashes per second the bitcoin network is performing in the last 24 hours.
            "difficulty", # Network Difficulty: A relative measure of how difficult it is to mine a new block for the blockchain.
            "miners-revenue", # Miners Revenue (USD): Total value of coinbase block rewards and transaction fees paid to miners.
            "transaction-fees-usd", # Total Transaction Fees (USD): The total USD value of all transaction fees paid to miners. This does not include coinbase block rewards.

          ##Network Activity##
            "n-unique-addresses", # The total number of unique addresses used on the blockchain.
            "n-transactions", # Confirmed Transactions Per Day: The total number of confirmed transactions per day.
            "estimated-transaction-volume-usd" # Estimated Transaction Value (USD): The total estimated value in USD of transactions on the blockchain. This does not include coins returned as change.
]

# Retreiving data

In this section we are going to make the call to the Blockchain.com API to retrieve the data.

In [21]:
def data_crawler(timespan, metrics, start_date, continue_date, end_date):
    # API Info
    url1 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={start_date}&format=csv'
    url2 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={continue_date}&format=csv'

    # Obtain Data
    data1 = pd.read_csv(url1,names=['timestamp',metrics])
    data2 = pd.read_csv(url2,names=['timestamp',metrics])

    # Concat by rows
    all_data = pd.concat([data1,data2])

    # Transform "timestamp" to datetime type
    all_data['timestamp'] = pd.to_datetime(all_data["timestamp"])

    # Keep the same end date with Bitcoin data
    all_data = all_data[(all_data['timestamp'] < end_date)]

    return all_data

In [22]:
# Merge the data
merge = functools.partial(pd.merge, on='timestamp')

# Gain Blockchain Data from Blockchain.com API
df1 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df1

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-03,432.76,1.287046e+06,54754.876205,0.554656,101427625,983.503448,7.694240e+05,1.038803e+11,1.720316e+06,11009.108568,335666.0,142608.0,6.764693e+07
3,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
4,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,2023-07-26,29226.18,7.731961e+07,498719.595538,1.682859,869881904,3144.069444,3.811901e+08,5.325165e+13,2.692680e+07,551760.191132,737067.0,452746.0,2.649786e+09
2757,2023-07-27,29344.56,1.112411e+08,498961.880465,1.653488,870335062,2993.028986,3.589731e+08,5.232831e+13,2.634722e+07,619524.768989,713630.0,413038.0,3.329622e+09
2758,2023-07-28,29213.94,8.336315e+07,499189.833530,1.629711,870747604,3208.750000,3.641756e+08,5.232831e+13,2.623381e+07,596803.108800,720574.0,449225.0,2.970637e+09
2759,2023-07-29,29316.12,7.587452e+07,499418.145973,1.616703,871197669,3205.446970,3.433656e+08,5.232831e+13,2.470638e+07,474147.692736,694496.0,423119.0,1.601999e+09


In [23]:
# Check duplicated rows
len(df1['timestamp'].unique())

2761

Due to a problem with the Blockchain.com API, I was forced to make an additional call to retrieve capitalization and total circulating data that will be added to the currency statistics to get a single dataset.

In [24]:
# Retrieving market capitalization and total circulating data
metrics = [
  "total-bitcoins", # Total Circulating Bitcoin: The total number of mined bitcoin that are currently circulating on the network.
  "market-cap", # Market Capitalization (USD): The total USD value of bitcoin in circulation.
  ]

merge = functools.partial(pd.merge, on='timestamp')
df2 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01 00:03:21,15029575.00,6.474140e+09
1,2016-01-02 15:54:15,15035125.00,6.499685e+09
2,2016-01-04 02:26:37,15040650.00,6.458253e+09
3,2016-01-05 09:50:06,15046150.00,6.489359e+09
4,2016-01-06 21:03:44,15051750.00,6.472554e+09
...,...,...,...
2979,2023-07-29 04:38:58,19442062.50,5.722577e+11
2980,2023-07-29 15:36:00,19442418.75,5.703045e+11
2981,2023-07-30 03:04:31,19442775.00,5.707815e+11
2982,2023-07-30 12:51:00,19443131.25,5.707920e+11


In [25]:
# Check duplicated rows
len(df2['timestamp'].unique())

2984

In [26]:
# Wipe off the timestamp's h:m:s.
df2['timestamp'] = pd.to_datetime(df2["timestamp"]).dt.normalize()

# Drop the duplicates in column "timestamp", keep the last value
df2.drop_duplicates(subset="timestamp", keep="last", inplace=True)

In [27]:
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01,15029575.00,6.474140e+09
1,2016-01-02,15035125.00,6.499685e+09
2,2016-01-04,15040650.00,6.458253e+09
3,2016-01-05,15046150.00,6.489359e+09
4,2016-01-06,15051750.00,6.472554e+09
...,...,...,...
2973,2023-07-26,19439918.75,5.704255e+11
2975,2023-07-27,19440631.25,5.712824e+11
2978,2023-07-28,19441706.25,5.704197e+11
2980,2023-07-29,19442418.75,5.703045e+11


In [28]:
# Check duplicated rows
len(df2['timestamp'].unique())

2081

In [29]:
# Add the market capitalization and total circulating data
all_data = pd.merge(df1, df2, how="left", on='timestamp')
all_data = all_data.interpolate(method='ffill')
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-03,432.76,1.287046e+06,54754.876205,0.554656,101427625,983.503448,7.694240e+05,1.038803e+11,1.720316e+06,11009.108568,335666.0,142608.0,6.764693e+07,15035125.00,6.499685e+09
3,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
4,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,2023-07-26,29226.18,7.731961e+07,498719.595538,1.682859,869881904,3144.069444,3.811901e+08,5.325165e+13,2.692680e+07,551760.191132,737067.0,452746.0,2.649786e+09,19439918.75,5.704255e+11
2757,2023-07-27,29344.56,1.112411e+08,498961.880465,1.653488,870335062,2993.028986,3.589731e+08,5.232831e+13,2.634722e+07,619524.768989,713630.0,413038.0,3.329622e+09,19440631.25,5.712824e+11
2758,2023-07-28,29213.94,8.336315e+07,499189.833530,1.629711,870747604,3208.750000,3.641756e+08,5.232831e+13,2.623381e+07,596803.108800,720574.0,449225.0,2.970637e+09,19441706.25,5.704197e+11
2759,2023-07-29,29316.12,7.587452e+07,499418.145973,1.616703,871197669,3205.446970,3.433656e+08,5.232831e+13,2.470638e+07,474147.692736,694496.0,423119.0,1.601999e+09,19442418.75,5.703045e+11


In [30]:
# Check nan value
all_data[all_data.isnull().T.any()]

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap


In [31]:
# Check duplicated rows
len(all_data['timestamp'].unique())

2761

In [32]:
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-03,432.76,1.287046e+06,54754.876205,0.554656,101427625,983.503448,7.694240e+05,1.038803e+11,1.720316e+06,11009.108568,335666.0,142608.0,6.764693e+07,15035125.00,6.499685e+09
3,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
4,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,2023-07-26,29226.18,7.731961e+07,498719.595538,1.682859,869881904,3144.069444,3.811901e+08,5.325165e+13,2.692680e+07,551760.191132,737067.0,452746.0,2.649786e+09,19439918.75,5.704255e+11
2757,2023-07-27,29344.56,1.112411e+08,498961.880465,1.653488,870335062,2993.028986,3.589731e+08,5.232831e+13,2.634722e+07,619524.768989,713630.0,413038.0,3.329622e+09,19440631.25,5.712824e+11
2758,2023-07-28,29213.94,8.336315e+07,499189.833530,1.629711,870747604,3208.750000,3.641756e+08,5.232831e+13,2.623381e+07,596803.108800,720574.0,449225.0,2.970637e+09,19441706.25,5.704197e+11
2759,2023-07-29,29316.12,7.587452e+07,499418.145973,1.616703,871197669,3205.446970,3.433656e+08,5.232831e+13,2.470638e+07,474147.692736,694496.0,423119.0,1.601999e+09,19442418.75,5.703045e+11


In [33]:
def move_columns(dataset, target_colum, column_to_move):
  cols = list(dataset.columns)
  cols.remove(column_to_move)
  cols.insert(cols.index(target_colum)+1, column_to_move)
  dataset = dataset.reindex(columns=cols)

  return dataset

In [34]:
# Move the column 'total-bitcoins' and 'market-cap' after the column 'market-price'
all_data = move_columns(all_data, 'market-price', 'total-bitcoins')
all_data = move_columns(all_data, 'market-price', 'market-cap')
all_data

,timestamp,market-price,market-cap,total-bitcoins,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,6.474140e+09,15029575.00,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,6.499685e+09,15035125.00,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-03,432.76,6.499685e+09,15035125.00,1.287046e+06,54754.876205,0.554656,101427625,983.503448,7.694240e+05,1.038803e+11,1.720316e+06,11009.108568,335666.0,142608.0,6.764693e+07
3,2016-01-04,430.78,6.458253e+09,15040650.00,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
4,2016-01-05,434.17,6.489359e+09,15046150.00,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,2023-07-26,29226.18,5.704255e+11,19439918.75,7.731961e+07,498719.595538,1.682859,869881904,3144.069444,3.811901e+08,5.325165e+13,2.692680e+07,551760.191132,737067.0,452746.0,2.649786e+09
2757,2023-07-27,29344.56,5.712824e+11,19440631.25,1.112411e+08,498961.880465,1.653488,870335062,2993.028986,3.589731e+08,5.232831e+13,2.634722e+07,619524.768989,713630.0,413038.0,3.329622e+09
2758,2023-07-28,29213.94,5.704197e+11,19441706.25,8.336315e+07,499189.833530,1.629711,870747604,3208.750000,3.641756e+08,5.232831e+13,2.623381e+07,596803.108800,720574.0,449225.0,2.970637e+09
2759,2023-07-29,29316.12,5.703045e+11,19442418.75,7.587452e+07,499418.145973,1.616703,871197669,3205.446970,3.433656e+08,5.232831e+13,2.470638e+07,474147.692736,694496.0,423119.0,1.601999e+09


Once we have the daily dataset we will go to sample it at a frequency of 1 minute (1T) using the resample method. This means that the data will be organized in 1-minute time-frame, and an interpolation method will be used to fill in any missing data or holes in the DataFrame by estimating missing values based on the surrounding known values.

In [35]:
# Upsampling to 1min by interpolate
all_data.set_index('timestamp', inplace=True)
all_data_1h = all_data.resample('1H').interpolate()
all_data_1h

,market-price,market-cap,total-bitcoins,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
timestamp,,,,,,,,,,,,,,,
2016-01-01 00:00:00,430.890000,6.474140e+09,1.502958e+07,2.860153e+06,54604.791735,0.493407,1.011557e+08,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.000000,124092.000000,5.834626e+07
2016-01-01 01:00:00,431.050833,6.475204e+09,1.502981e+07,2.809565e+06,54607.538237,0.497004,1.011608e+08,923.727011,6.975595e+05,1.038803e+11,1.559629e+06,8992.039015,266036.875000,125131.416667,5.803318e+07
2016-01-01 02:00:00,431.211667,6.476268e+09,1.503004e+07,2.758977e+06,54610.284739,0.500602,1.011659e+08,928.254023,6.979898e+05,1.038803e+11,1.564490e+06,9201.014178,268952.750000,126170.833333,5.772010e+07
2016-01-01 03:00:00,431.372500,6.477333e+09,1.503027e+07,2.708389e+06,54613.031241,0.504199,1.011710e+08,932.781034,6.984201e+05,1.038803e+11,1.569350e+06,9409.989342,271868.625000,127210.250000,5.740702e+07
2016-01-01 04:00:00,431.533333,6.478397e+09,1.503050e+07,2.657801e+06,54615.777742,0.507797,1.011761e+08,937.308046,6.988505e+05,1.038803e+11,1.574211e+06,9618.964505,274784.500000,128249.666667,5.709394e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-29 20:00:00,29352.595000,5.688077e+11,1.944331e+07,3.926979e+07,499595.925800,1.704247,8.715501e+08,3737.882035,4.214032e+08,5.232831e+13,3.052827e+07,503742.351307,782832.666667,608731.500000,1.421208e+09
2023-07-29 21:00:00,29354.418750,5.687328e+11,1.944336e+07,3.743955e+07,499604.814792,1.708625,8.715677e+08,3764.503788,4.253051e+08,5.232831e+13,3.081936e+07,505222.084235,787249.500000,618012.125000,1.412168e+09
2023-07-29 22:00:00,29356.242500,5.686580e+11,1.944340e+07,3.560932e+07,499613.703783,1.713002,8.715854e+08,3791.125541,4.292070e+08,5.232831e+13,3.111045e+07,506701.817164,791666.333333,627292.750000,1.403129e+09


# Output

In this last section we are going to save the dataset we just created to the Google Drive.

In [36]:
GDRIVE_DIR = "/content/drive"
GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"

In [37]:
# Link Colab to our Google Drive
drive.mount(GDRIVE_DIR)

Mounted at /content/drive


In [38]:
def output(dataset, path):
  dataset.to_parquet(path)

In [39]:
# Output the 1h data
GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_1h"
GDRIVE_DATASET_NAME_EXT = "/" + GDRIVE_DATASET_NAME + ".parquet"
GDRIVE_DATASET = GDRIVE_DATASET_RAW_DIR + GDRIVE_DATASET_NAME_EXT
output(all_data_1h, GDRIVE_DATASET)